# Practice: Speech

## Learn Demo

In [ ]:
import os
import azure.cognitiveservices.speech as speechsdk
from dotenv import load_dotenv

load_dotenv()

def recognize_from_microphone():
    # This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
    speech_config = speechsdk.SpeechConfig(subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
    speech_config.speech_recognition_language="en-US"

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("Speak into your microphone.")
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print("Recognized: {}".format(speech_recognition_result.text))
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print("No speech could be recognized: {}".format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print("Speech Recognition canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

recognize_from_microphone()

Speak into your microphone.
Recognized: Hello. Hello. Hello. Hello. Hello. Hello.


## Practice: Speech

### Requirements:

- Transcribe voice to text (Speech) - show the transcribed text 

- Summary the call  - Show summary 

- Audio file: BillGates_2010.wav 

### Phase 1
1. Create Speech Service & Language Service

### Phase 2
1. Coding

1.1 Set up environment

In [64]:
import os
import time
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

load_dotenv()
document = []

1.2 Recognize the wav file
- Step1: Instant a speech SDK, and set up configs
- Step2: Connect callbacks to the event, create document list to store the recognized text.
- Step3: Start to recognize the speech file and store the result in document list.

In [65]:
def recognize_speech_to_text():
    audio_file = "/Users/lawrence/Work/AI/azure_ai/Practices/w1_speech/BillGates_2010.wav"
    speech_config = speechsdk.SpeechConfig(subscription=os.environ.get("SPEECH_KEY"), region=os.environ.get("SPEECH_REGION"))
    speech_config.speech_recognition_language = "en-US"
    speech_config.set_property(property_id=speechsdk.properties.PropertyId.SpeechServiceConnection_InitialSilenceTimeoutMs, value="450000")
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    def stop_cb(evt):
        print("Closing on {}".format(evt))
        speech_recognizer.stop_continuous_recognition()
        nonlocal done
        done = True

    def create_doc(evt):
        print("Recognized: {}".format(evt))
        print("Text: {}".format(evt.result.text))
        document.append(evt.result.text)
    
    # Connect callbacks to the events fired by the speech recognizer
    # speech_recognizer.recognizing.connect(lambda evt: print("Recognizing: {}".format(evt)))
    speech_recognizer.recognized.connect(create_doc)
    speech_recognizer.session_started.connect(lambda evt: print("Session Started: {}".format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print("Session Stopped: {}".format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print("Canceled: {}".format(evt)))
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)
    document.append(doc_text)
    print(document)
    


recognize_speech_to_text()

Session Started: SessionEventArgs(session_id=8442f67043704769bfb31965a1323567)
Recognized: SpeechRecognitionEventArgs(session_id=8442f67043704769bfb31965a1323567, result=SpeechRecognitionResult(result_id=70118600cdd248218996735970ad87dd, text="I'm going to talk today about energy and climate, and that might seem a bit surprising because my full time work at the foundation is mostly about vaccines and seeds, about the things that we need to invent and deliver to help the poorest 2 billion live better lives. But energy and climate are extremely important to these people, in fact, more important than to anyone else on the planet.", reason=ResultReason.RecognizedSpeech))
Text: I'm going to talk today about energy and climate, and that might seem a bit surprising because my full time work at the foundation is mostly about vaccines and seeds, about the things that we need to invent and deliver to help the poorest 2 billion live better lives. But energy and climate are extremely important to 

1.3 Summarize the document
- Step 1: Transfer the document list to one String and store in a list.
- Step 2: Summarize the document.

In [66]:
def summarize_text():
    endpoint = os.environ.get("AZURE_LANGUAGE_ENDPOINT")
    key = os.environ.get("AZURE_LANGUAGE_KEY")

    text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))

    print(document)
    summarized_list = []
    doc_size = len(document)

    # for i in range(int(doc_size/24) + 1):
    #     doc_text = ""
    #     if (i+1)*24 < doc_size-1: 
    #         new_doc_list = document[i*24:(i+1)*24]
    #     else:
    #         new_doc_list = document[i*24:doc_size-1]
    #     poller = text_analytics_client.begin_abstract_summary(new_doc_list)
    #     abstract_summary_results = poller.result()

    #     for result in abstract_summary_results:
    #         if result.kind == "AbstractiveSummarization":
    #             for summary in result.summaries:
    #                 doc_text = doc_text + " " + summary.text

    #         elif result.is_error is True:
    #             print("...Is an error with code '{}' and message '{}'".format(
    #                 result.error.code, result.error.message
    #             ))
    #     summarized_list.append(doc_text)

    doc_text = ""
    for sentence in document:
        doc_text = doc_text + " " + sentence
    print(doc_text)
    summarized_list.append(doc_text) 
    
    poller = text_analytics_client.begin_abstract_summary(summarized_list)
    abstract_summary_results = poller.result()
    for result in abstract_summary_results:
        if result.kind == "AbstractiveSummarization":
            print("Summaries abstracted:")
            [print(f"{summary.text}\n") for summary in result.summaries]
        elif result.is_error is True:
            print("...Is an error with code '{}' and message '{}'".format(
                result.error.code, result.error.message
            ))

summarize_text()

["I'm going to talk today about energy and climate, and that might seem a bit surprising because my full time work at the foundation is mostly about vaccines and seeds, about the things that we need to invent and deliver to help the poorest 2 billion live better lives. But energy and climate are extremely important to these people, in fact, more important than to anyone else on the planet.", "The climate getting worse means that many years their crops won't grow, there'll be too much rain, not enough rain. Things will change in ways that their fragile environment simply can't support. And that leads to starvation, that leads to uncertainty, it leads to unrest. Also the the climate changes will be terrible for them. Also, the price of energy is very important to them. In fact, if you could pick just one thing to lower the price of to reduce poverty.", "By far, you would pick energy. Now, the price of energy has come down over time. Really advanced civilization is based on advances in in